In [1]:
!pip install joblib
!conda install --yes joblib

Defaulting to user installation because normal site-packages is not writeable
Solving environment: done

## Package Plan ##

  environment location: /opt/intel/oneapi/intelpython/latest

  added / updated specs:
    - joblib


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    anaconda::ca-certificates-2023.08.22-~ --> conda_channel::ca-certificates-2023.5.7-hbcca054_0 
  certifi            anaconda/linux-64::certifi-2023.7.22-~ --> conda_channel/noarch::certifi-2023.5.7-pyhd8ed1ab_0 




Preparing transaction: done
Verifying transaction: failed

EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment location: /opt/intel/oneapi/intelpython/latest
  uid: 17789
  gid: 100




In [2]:
import joblib

In [3]:
!pip install --upgrade scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold,cross_val_score,GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report, precision_score, roc_curve


import seaborn as sns
from sklearn.utils import shuffle

from sklearn.linear_model import LogisticRegression, Perceptron, RidgeClassifier, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier 
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, VotingClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

**Read and shuffle the dataset**

In [5]:
df = pd.read_csv('dataset.csv')
df = shuffle(df,random_state=42)

**Removing Hyphen from strings**

In [6]:
for col in df.columns:
    df[col] = df[col].str.replace('_',' ')

**Remove the trailing space from the symptom columns**

In [7]:
cols = df.columns
data = df[cols].values.flatten()

s = pd.Series(data)
s = s.str.strip()
s = s.values.reshape(df.shape)

df = pd.DataFrame(s, columns=df.columns)


**Fill the NaN values with zero**

In [8]:
df = df.fillna(0)

**Symptom severity rank**

In [9]:
df1 = pd.read_csv('Symptom-severity.csv')
df1['Symptom'] = df1['Symptom'].str.replace('_',' ')

**Get overall list of symptoms**

In [10]:
#df1['Symptom'].unique()

**Encode symptoms in the data with the symptom rank**

In [11]:
vals = df.values
symptoms = df1['Symptom'].unique()

for i in range(len(symptoms)):
    vals[vals == symptoms[i]] = df1[df1['Symptom'] == symptoms[i]]['weight'].values[0]
    
d = pd.DataFrame(vals, columns=cols)


**Assign symptoms with no rank to zero**

In [12]:
d = d.replace('dischromic  patches', 0)
d = d.replace('spotting  urination',0)
df = d.replace('foul smell of urine',0)

In [13]:
print("Number of symptoms used to identify the disease ",len(df1['Symptom'].unique()))
print("Number of diseases that can be identified ",len(df['Disease'].unique()))

Number of symptoms used to identify the disease  132
Number of diseases that can be identified  41


In [14]:
data = df.iloc[:,1:].values
labels = df['Disease'].values

<h2>Training and Testing Split</h2>

In [15]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, train_size = 0.8,random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(3936, 17) (984, 17) (3936,) (984,)


# Decision Tree

In [16]:
tree =DecisionTreeClassifier(criterion='gini',random_state=42,max_depth=13)
tree.fit(x_train, y_train)
preds=tree.predict(x_test)
print('F1-score% =', f1_score(y_test, preds, average='macro')*100, '|', 'Accuracy% =', accuracy_score(y_test, preds)*100)



F1-score% = 95.7812664665753 | Accuracy% = 95.83333333333334


In [17]:
kfold = KFold(n_splits=10,shuffle=True,random_state=42)
DS_train =cross_val_score(tree, x_train, y_train, cv=kfold, scoring='accuracy')
pd.DataFrame(DS_train,columns=['Scores'])
print("Mean Accuracy: %.3f%%, Standard Deviation: (%.2f%%)" % (DS_train.mean()*100.0, DS_train.std()*100.0))

Mean Accuracy: 95.706%, Standard Deviation: (1.99%)


In [18]:
kfold = KFold(n_splits=10,shuffle=True,random_state=42)
DS_test =cross_val_score(tree, x_test, y_test, cv=kfold, scoring='accuracy')
pd.DataFrame(DS_test,columns=['Scores'])
print("Mean Accuracy: %.3f%%, Standard Deviation: (%.2f%%)" % (DS_test.mean()*100.0, DS_test.std()*100.0))

Mean Accuracy: 94.198%, Standard Deviation: (3.57%)


# Random Forest

In [19]:
rfc=RandomForestClassifier(random_state=42)

In [20]:
rfc = RandomForestClassifier(random_state=42, max_features='sqrt', n_estimators= 500, max_depth=13)
rfc.fit(x_train,y_train)
preds= rfc.predict(x_test)
print(x_test[0])
print(preds[0])
conf_mat = confusion_matrix(y_test, preds)
df_cm = pd.DataFrame(conf_mat, index=df['Disease'].unique(), columns=df['Disease'].unique())
print('F1-score% =', f1_score(y_test, preds, average='macro')*100, '|', 'Accuracy% =', accuracy_score(y_test, preds)*100)


[3 5 3 5 4 4 3 2 3 0 0 0 0 0 0 0 0]
Migraine
F1-score% = 99.58380389536958 | Accuracy% = 99.59349593495935


In [21]:
kfold = KFold(n_splits=10,shuffle=True,random_state=42)
rnd_forest_train =cross_val_score(rfc, x_train, y_train, cv=kfold, scoring='accuracy')
pd.DataFrame(rnd_forest_train,columns=['Scores'])
print("Mean Accuracy: %.3f%%, Standard Deviation: (%.2f%%)" % (rnd_forest_train.mean()*100.0, rnd_forest_train.std()*100.0))

Mean Accuracy: 99.187%, Standard Deviation: (0.44%)


In [22]:
kfold = KFold(n_splits=10,shuffle=True,random_state=42)
rnd_forest_test =cross_val_score(rfc, x_test, y_test, cv=kfold, scoring='accuracy')
pd.DataFrame(rnd_forest_test,columns=['Scores'])
print("Mean Accuracy: %.3f%%, Standard Deviation: (%.2f%%)" % (rnd_forest_test.mean()*100.0, rnd_forest_test.std()*100.0))

Mean Accuracy: 98.575%, Standard Deviation: (1.31%)


In [23]:
discrp = pd.read_csv("symptom_Description.csv")

In [24]:
ektra7at = pd.read_csv("symptom_precaution.csv")

In [25]:
def predd(x, s1=0.0, s2=0.0, s3=0.0, s4=0.0, s5=0.0, s6=0.0, s7=0.0, s8=0.0, s9=0.0, s10=0.0,
          s11=0.0, s12=0.0, s13=0.0, s14=0.0, s15=0.0, s16=0.0, s17=0.0):
    psymptoms = [s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12, s13, s14, s15, s16, s17]

    # Update psymptoms with weights from the dataframe
    a = np.array(df1["Symptom"])
    b = np.array(df1["weight"])
    for j in range(len(psymptoms)):
        for k in range(len(a)):
            if psymptoms[j] == a[k]:
                psymptoms[j] = b[k]

    # Predict disease using the provided RandomForestClassifier instance
    psy = [psymptoms]
    pred2 = x.predict(psy)

    # Get top 3 probabilities
    probas = x.predict_proba(psy)[0]
    top_indices = probas.argsort()[::-1][:3]
    top_diseases = x.classes_[top_indices]
    top_probas = probas[top_indices]

    # Print top 3 predicted diseases with descriptions and precaution lists
    for disease, proba in zip(top_diseases, top_probas):
        disp = discrp[discrp['Disease'] == disease].values[0][1]
        recomnd = ektra7at[ektra7at['Disease'] == disease]
        c = np.where(ektra7at['Disease'] == disease)[0][0]
        precuation_list = list(ektra7at.iloc[c, 1:])

        print("Potential Disease Name:", disease)
        print("Probability: {:.2f}%".format(proba * 100))
        print("Description:", disp)
        print("Recommended Things to do at home:")
        for i in precuation_list:
            print("-", i)
        print()  # Add an empty line for better readability


In [26]:
symptoms = ['itching', 'skin_rash', 'nodal_skin_eruptions',
       'continuous_sneezing', 'shivering', 'chills', 'joint_pain',
       'stomach_pain', 'acidity', 'ulcers_on_tongue', 'muscle_wasting',
       'vomiting', 'burning_micturition', 'spotting_urination', 'fatigue',
       'weight_gain', 'anxiety', 'cold_hands_and_feets', 'mood_swings',
       'weight_loss', 'restlessness', 'lethargy', 'patches_in_throat',
       'irregular_sugar_level', 'cough', 'high_fever', 'sunken_eyes',
       'breathlessness', 'sweating', 'dehydration', 'indigestion',
       'headache', 'yellowish_skin', 'dark_urine', 'nausea',
       'loss_of_appetite', 'pain_behind_the_eyes', 'back_pain',
       'constipation', 'abdominal_pain', 'diarrhoea', 'mild_fever',
       'yellow_urine', 'yellowing_of_eyes', 'acute_liver_failure',
       'fluid_overload', 'swelling_of_stomach', 'swelled_lymph_nodes',
       'malaise', 'blurred_and_distorted_vision', 'phlegm',
       'throat_irritation', 'redness_of_eyes', 'sinus_pressure',
       'runny_nose', 'congestion', 'chest_pain', 'weakness_in_limbs',
       'fast_heart_rate', 'pain_during_bowel_movements',
       'pain_in_anal_region', 'bloody_stool', 'irritation_in_anus',
       'neck_pain', 'dizziness', 'cramps', 'bruising', 'obesity',
       'swollen_legs', 'swollen_blood_vessels', 'puffy_face_and_eyes',
       'enlarged_thyroid', 'brittle_nails', 'swollen_extremeties',
       'excessive_hunger', 'extra_marital_contacts',
       'drying_and_tingling_lips', 'slurred_speech', 'knee_pain',
       'hip_joint_pain', 'muscle_weakness', 'stiff_neck',
       'swelling_joints', 'movement_stiffness', 'spinning_movements',
       'loss_of_balance', 'unsteadiness', 'weakness_of_one_body_side',
       'loss_of_smell', 'bladder_discomfort', 'foul_smell_ofurine',
       'continuous_feel_of_urine', 'passage_of_gases', 'internal_itching',
       'toxic_look_(typhos)', 'depression', 'irritability', 'muscle_pain',
       'altered_sensorium', 'red_spots_over_body', 'belly_pain',
       'abnormal_menstruation', 'dischromic_patches',
       'watering_from_eyes', 'increased_appetite', 'polyuria',
       'family_history', 'mucoid_sputum', 'rusty_sputum',
       'lack_of_concentration', 'visual_disturbances',
       'receiving_blood_transfusion', 'receiving_unsterile_injections',
       'coma', 'stomach_bleeding', 'distention_of_abdomen',
       'history_of_alcohol_consumption', 'blood_in_sputum',
       'prominent_veins_on_calf', 'palpitations', 'painful_walking',
       'pus_filled_pimples', 'blackheads', 'scurring', 'skin_peeling',
       'silver_like_dusting', 'small_dents_in_nails',
       'inflammatory_nails', 'blister', 'red_sore_around_nose',
       'yellow_crust_ooze', 'prognosis']

In [27]:
def preprocess_input(input_text):
    tokens = input_text.lower().split()
    return tokens

def identify_symptoms(tokens):
    # Initialize a list to store identified symptoms
    identified_symptoms = []
    # Check each token against the list of symptoms
    for token in tokens:
        # Use a regular expression to check for partial matches
        if any(re.search(r'\b{}\b'.format(symptom), token) for symptom in symptoms):
            identified_symptoms.append(token)
    return identified_symptoms

def predict_disease(identified_symptoms):
    print("Identified Symptoms: ", identified_symptoms)
    print()
    prediction = predd(rfc, *identified_symptoms)
    return prediction


In [28]:
def chatbot():
    print("Welcome to the Disease Predictor Chatbot!")
    print("You can describe your symptoms, and I'll try to predict the possible disease.")
    print("Type 'exit' to end the conversation.")

    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        else:
            tokens = preprocess_input(user_input)
            identified_symptoms = identify_symptoms(tokens)
            if identified_symptoms:
                predicted_disease = predict_disease(identified_symptoms)
                print("Chatbot: It appears that your symptoms match the disease listed above. However, please keep in mind that I am still in development. For accurate diagnosis and treatment, please consult a healthcare professional.")
            else:
                print("Chatbot: No symptoms identified. I am still a bot that is being trained and tested, so I might not recognize all diseases and symptoms accurately, please consult your nearest doctor about your symptoms. If you'd like to exit, please type 'exit'.")



In [ ]:
chatbot()

Welcome to the Disease Predictor Chatbot!
You can describe your symptoms, and I'll try to predict the possible disease.
Type 'exit' to end the conversation.
